In [1]:
import pandas as  pd
import pymongo
import requests
import json
from datetime import datetime
import time
import calendar

In [ ]:
bike_data_csv = "data/metro-bike-share-trips-2018-q3.csv"

In [ ]:
bike_data_df = pd.read_csv(bike_data_csv)

In [ ]:
bike_data_df['weekday'] = bike_data_df.apply(lambda row: calendar.day_name[datetime.strptime(row.start_time, '%Y-%m-%d %H:%M:%S').weekday()] , axis=1)

In [ ]:
bike_data_df['start_day'] = bike_data_df.apply(lambda row: datetime.strptime(row.start_time, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') , axis=1)

In [ ]:
bike_data_df.head()                           

In [ ]:
bike_data_df['start_time'] = pd.to_datetime(bike_data_df['start_time'], format='%Y-%m-%d %H:%M:%S')
bike_data_df['end_time'] = pd.to_datetime(bike_data_df['end_time'], format='%Y-%m-%d %H:%M:%S')  

In [ ]:
#items = bike_data_df[:1000].to_json(orient='records', date_format='iso', date_unit='s')
items = bike_data_df[:1000].to_json(orient='records')

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017/bike_data_db'
client = pymongo.MongoClient(conn)

In [10]:
db = client.bike_data_db
collection = db.bike_trip

In [11]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bike_data_db'), 'bike_trip')

In [ ]:
items_db = json.loads(items)

In [ ]:
collection.drop()

In [ ]:
db.bike_trip.insert_many(items_db)

In [12]:
collection.count_documents({})

93199

In [ ]:
collection.distinct("start_day")

In [8]:
collection.find_one()

TypeError: 'Collection' object is not callable. If you meant to call the 'find_one' method on a 'Database' object it is failing because no such method exists.

In [ ]:
oneWayDoc = collection.find({"trip_route_category": "One Way"})

In [ ]:
oneWayDoc[0]

In [ ]:
#temp = collection.find({"start_time": {'$gte': '2018-07-01T00:00:00Z', '$lt': '2018-07-31T23:59:59Z'}})
temp = collection.find({"start_day": {'$gte': '2018-07-01', '$lt': '2018-07-31'}})

In [ ]:
list(temp)

In [ ]:
datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%S.%fZ")

In [ ]:
from flask import jsonify

In [ ]:
bike = list(temp)

In [ ]:
bike

In [ ]:
for x in bike:
    x.pop('_id', None)
bike[0]

In [ ]:
b_link = "http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v5.geojson"

In [13]:
collection = db.la_boundary

In [14]:
collection.find()

In [ ]:
r = requests.get(b_link)

In [ ]:
r.json()

In [ ]:
collection.insert(r.json())

In [ ]:
temp = collection.find()

In [ ]:
temp = list(temp)

In [ ]:
for i in temp:
    i.pop('_id', None)

In [4]:
db = client

In [3]:
db = client.bike_rental

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bike_rental')

In [6]:
collection = db.bike_rental

In [7]:
collection.count_documents({})

0

In [9]:
db = client.bike_data_db

bike_trip = db.bike_trip.find()

full_dict = []
for trip in bike_trip:
    full_dict.append(trip)
df = pd.DataFrame(full_dict)

# use k to filter dataframe based on passholder type
# group by day of week and pass info back into d3

#pass_type = request.args.get('pass_type')

#selection = df.loc[df['passholder_type'] == pass_type, :]    
#grouped_df = selection[['weekday','duration']].groupby('weekday').sum()
#index_reset = grouped_df.reset_index()
#index_reset['weekday'] = pd.Categorical(index_reset['weekday'], categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], ordered=True)
#weekday_df = index_reset.sort_values('weekday')




In [13]:
df.head().groupby(['passholder_type', 'weekday']).sum()

,,bike_id,duration,end_lat,end_lon,end_station,plan_duration,start_lat,start_lon,start_station,trip_id
passholder_type,weekday,,,,,,,,,,
Flex Pass,Sunday,6518,8,34.046520,-118.237411,3082,365,34.035801,-118.233170,3058,94851141
Monthly Pass,Sunday,17217,30,102.359310,-354.536391,11418,425,102.322048,-354.515328,11362,284553415
Walk-up,Sunday,12055,35,33.779819,-118.263023,3013,1,33.779819,-118.263023,3013,94851136


In [49]:
grouped_df = df[['passholder_type','weekday', 'duration']].groupby(['passholder_type','weekday']).sum()

In [48]:
group2_df

[Saturday, Sunday, Friday, Monday, Saturday, ..., Saturday, Sunday, Thursday, Tuesday, Wednesday]
Length: 30
Categories (7, object): [Monday < Tuesday < Wednesday < Thursday < Friday < Saturday < Sunday]

In [50]:
group2_df = grouped_df.reset_index()

In [46]:
#group2_df = pd.Categorical(group2_df['weekday'], categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], ordered=True)

In [51]:
item_db = dict()
for index, row in group2_df.iterrows():
    print(row['passholder_type'])
    key = row['passholder_type']
    day = row['weekday']
    duration = row['duration']
    if (key in item_db):
        item_db[key].append({'weekday': day, 'duration': duration})
    else:
        item_db[key] = [{'weekday': day, 'duration': duration}]

Annual Pass
Annual Pass
Flex Pass
Flex Pass
Flex Pass
Flex Pass
Flex Pass
Flex Pass
Flex Pass
Monthly Pass
Monthly Pass
Monthly Pass
Monthly Pass
Monthly Pass
Monthly Pass
Monthly Pass
One Day Pass
One Day Pass
One Day Pass
One Day Pass
One Day Pass
One Day Pass
One Day Pass
Walk-up
Walk-up
Walk-up
Walk-up
Walk-up
Walk-up
Walk-up


In [25]:
item_db

{'Annual Pass': [{'weekday': 'Saturday', 'duration': 242},
  {'weekday': 'Sunday', 'duration': 163}],
 'Flex Pass': [{'weekday': 'Friday', 'duration': 5391},
  {'weekday': 'Monday', 'duration': 4318},
  {'weekday': 'Saturday', 'duration': 4730},
  {'weekday': 'Sunday', 'duration': 5049},
  {'weekday': 'Thursday', 'duration': 3222},
  {'weekday': 'Tuesday', 'duration': 3676},
  {'weekday': 'Wednesday', 'duration': 2683}],
 'Monthly Pass': [{'weekday': 'Friday', 'duration': 105751},
  {'weekday': 'Monday', 'duration': 103022},
  {'weekday': 'Saturday', 'duration': 86178},
  {'weekday': 'Sunday', 'duration': 90774},
  {'weekday': 'Thursday', 'duration': 102579},
  {'weekday': 'Tuesday', 'duration': 99726},
  {'weekday': 'Wednesday', 'duration': 94277}],
 'One Day Pass': [{'weekday': 'Friday', 'duration': 37332},
  {'weekday': 'Monday', 'duration': 36064},
  {'weekday': 'Saturday', 'duration': 58287},
  {'weekday': 'Sunday', 'duration': 83727},
  {'weekday': 'Thursday', 'duration': 24697},

In [27]:
db.bike_rental.insert(item_db)

C:\Users\charl\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


ObjectId('5c0893488a95a61b086be51e')

In [28]:
item_db['One Day Pass']

[{'weekday': 'Friday', 'duration': 37332},
 {'weekday': 'Monday', 'duration': 36064},
 {'weekday': 'Saturday', 'duration': 58287},
 {'weekday': 'Sunday', 'duration': 83727},
 {'weekday': 'Thursday', 'duration': 24697},
 {'weekday': 'Tuesday', 'duration': 22184},
 {'weekday': 'Wednesday', 'duration': 33385}]

In [36]:
bike_trip = db.bike_rental.find()

In [37]:
bike_trip = list(bike_trip)

In [39]:
bike_trip[0]['Annual Pass']

[{'weekday': 'Saturday', 'duration': 242},
 {'weekday': 'Sunday', 'duration': 163}]